Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (197738, 28, 28) (197738,)
Validation set (9847, 28, 28) (9847,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197738, 28, 28, 1) (197738, 10)
Validation set (9847, 28, 28, 1) (9847, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.627468
Minibatch accuracy: 12.5%
Validation accuracy: 10.2%
Minibatch loss at step 50: 2.020308
Minibatch accuracy: 31.2%
Validation accuracy: 58.9%
Minibatch loss at step 100: 1.038943
Minibatch accuracy: 68.8%
Validation accuracy: 71.1%
Minibatch loss at step 150: 0.891136
Minibatch accuracy: 81.2%
Validation accuracy: 73.2%
Minibatch loss at step 200: 1.204124
Minibatch accuracy: 62.5%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.535137
Minibatch accuracy: 87.5%
Validation accuracy: 76.4%
Minibatch loss at step 300: 0.958479
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 350: 0.906165
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 400: 1.377204
Minibatch accuracy: 62.5%
Validation accuracy: 78.7%
Minibatch loss at step 450: 0.837917
Minibatch accuracy: 68.8%
Validation accuracy: 78.3%
Minibatch loss at step 500: 0.785997
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.390011
Minibatch accuracy: 6.2%
Validation accuracy: 10.1%
Minibatch loss at step 50: 2.322113
Minibatch accuracy: 6.2%
Validation accuracy: 17.0%
Minibatch loss at step 100: 1.675545
Minibatch accuracy: 31.2%
Validation accuracy: 39.0%
Minibatch loss at step 150: 1.048036
Minibatch accuracy: 75.0%
Validation accuracy: 64.9%
Minibatch loss at step 200: 1.108236
Minibatch accuracy: 68.8%
Validation accuracy: 69.7%
Minibatch loss at step 250: 0.671717
Minibatch accuracy: 81.2%
Validation accuracy: 76.2%
Minibatch loss at step 300: 0.733550
Minibatch accuracy: 81.2%
Validation accuracy: 76.8%
Minibatch loss at step 350: 0.806851
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 400: 0.907619
Minibatch accuracy: 62.5%
Validation accuracy: 79.6%
Minibatch loss at step 450: 0.719186
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.628684
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Min

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [89]:
batch_size = 128
patch_size = 5
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer_c1_weights = tf.get_variable("layer_c1_weights", 
      [patch_size, patch_size, num_channels, 32], initializer=tf.contrib.layers.xavier_initializer())
  layer_c1_biases = tf.Variable(tf.zeros([32]))
  layer_c3_weights = tf.get_variable("layer_c3_weights", 
      [patch_size, patch_size, 32, 64], initializer=tf.contrib.layers.xavier_initializer())
  layer_c3_biases = tf.Variable(tf.constant(0.1, shape=[64]))
  layer_c5_weights = tf.get_variable("layer_c5_weights",
      [patch_size*patch_size*64, 120], initializer=tf.contrib.layers.xavier_initializer())
  layer_c5_biases = tf.Variable(tf.constant(0.1, shape=[120]))
  layer_f6_weights = tf.get_variable("layer_f6_weights",
      [120, 84], initializer=tf.contrib.layers.xavier_initializer())
  layer_f6_biases = tf.Variable(tf.constant(0.1, shape=[84]))
  layer_output_weights = tf.get_variable("layer_output_weights",
      [84, 10], initializer=tf.contrib.layers.xavier_initializer())
  layer_output_biases = tf.Variable(tf.constant(.1, shape=[10]))

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer_c1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(hidden + layer_c1_biases)
    conv = tf.nn.conv2d(hidden, layer_c3_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(hidden + layer_c3_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    # print(reshape.get_shape())
    c5 = tf.matmul(reshape, layer_c5_weights) + layer_c5_biases
    c5 = tf.nn.relu(c5)
    f6 = tf.matmul(c5, layer_f6_weights) + layer_f6_biases
    hidden = tf.nn.relu(f6)
    return tf.matmul(hidden, layer_output_weights) + layer_output_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  regularizers = (tf.nn.l2_loss(layer_c1_weights) + tf.nn.l2_loss(layer_c1_biases) +
                  tf.nn.l2_loss(layer_c3_weights) + tf.nn.l2_loss(layer_c3_biases) +
                  tf.nn.l2_loss(layer_c5_weights) + tf.nn.l2_loss(layer_c5_biases) +
                  tf.nn.l2_loss(layer_f6_weights) + tf.nn.l2_loss(layer_f6_biases) + 
                  tf.nn.l2_loss(layer_output_weights) + tf.nn.l2_loss(layer_output_biases))
  print(regularizers)
  loss = 1e-9 * regularizers
  loss += tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Tensor("add_13:0", shape=(), dtype=float32)


In [91]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.339246
Minibatch accuracy: 9.4%
Validation accuracy: 13.6%
Minibatch loss at step 50: 0.872004
Minibatch accuracy: 75.8%
Validation accuracy: 74.3%
Minibatch loss at step 100: 0.490894
Minibatch accuracy: 85.2%
Validation accuracy: 79.4%
Minibatch loss at step 150: 0.689943
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Minibatch loss at step 200: 0.577497
Minibatch accuracy: 82.0%
Validation accuracy: 84.0%
Minibatch loss at step 250: 0.532399
Minibatch accuracy: 87.5%
Validation accuracy: 83.9%
Minibatch loss at step 300: 0.383150
Minibatch accuracy: 89.8%
Validation accuracy: 85.0%
Minibatch loss at step 350: 0.423812
Minibatch accuracy: 85.9%
Validation accuracy: 85.7%
Minibatch loss at step 400: 0.645947
Minibatch accuracy: 78.9%
Validation accuracy: 85.9%
Minibatch loss at step 450: 0.583083
Minibatch accuracy: 81.2%
Validation accuracy: 86.4%
Minibatch loss at step 500: 0.588830
Minibatch accuracy: 78.9%
Validation accuracy: 86.7%
Mi

- Adding Dropout

In [92]:
batch_size = 128
patch_size = 5
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer_c1_weights = tf.get_variable("layer_c1_weights", 
      [patch_size, patch_size, num_channels, 32], initializer=tf.contrib.layers.xavier_initializer())
  layer_c1_biases = tf.Variable(tf.zeros([32]))
  layer_c3_weights = tf.get_variable("layer_c3_weights", 
      [patch_size, patch_size, 32, 64], initializer=tf.contrib.layers.xavier_initializer())
  layer_c3_biases = tf.Variable(tf.constant(0.1, shape=[64]))
  layer_c5_weights = tf.get_variable("layer_c5_weights",
      [patch_size*patch_size*64, 120], initializer=tf.contrib.layers.xavier_initializer())
  layer_c5_biases = tf.Variable(tf.constant(0.1, shape=[120]))
  layer_f6_weights = tf.get_variable("layer_f6_weights",
      [120, 84], initializer=tf.contrib.layers.xavier_initializer())
  layer_f6_biases = tf.Variable(tf.constant(0.1, shape=[84]))
  layer_output_weights = tf.get_variable("layer_output_weights",
      [84, 10], initializer=tf.contrib.layers.xavier_initializer())
  layer_output_biases = tf.Variable(tf.constant(.1, shape=[10]))

  # Model.
  def model(data, dropout):
    conv = tf.nn.conv2d(data, layer_c1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(hidden + layer_c1_biases)
    if dropout:
      hidden = tf.nn.dropout(hidden, 0.5)
    conv = tf.nn.conv2d(hidden, layer_c3_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(hidden + layer_c3_biases)
    if dropout:
      hidden = tf.nn.dropout(hidden, 0.5)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    c5 = tf.matmul(reshape, layer_c5_weights) + layer_c5_biases
    c5 = tf.nn.relu(c5)
    if dropout:
      c5 = tf.nn.dropout(c5, 0.5)
    f6 = tf.matmul(c5, layer_f6_weights) + layer_f6_biases
    hidden = tf.nn.relu(f6)
    if dropout:
      hidden = tf.nn.dropout(hidden, 0.5)
    return tf.matmul(hidden, layer_output_weights) + layer_output_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 1)
  regularizers = (tf.nn.l2_loss(layer_c1_weights) + tf.nn.l2_loss(layer_c1_biases) +
                  tf.nn.l2_loss(layer_c3_weights) + tf.nn.l2_loss(layer_c3_biases) +
                  tf.nn.l2_loss(layer_c5_weights) + tf.nn.l2_loss(layer_c5_biases) +
                  tf.nn.l2_loss(layer_f6_weights) + tf.nn.l2_loss(layer_f6_biases) + 
                  tf.nn.l2_loss(layer_output_weights) + tf.nn.l2_loss(layer_output_biases))
  print(regularizers)
  loss = 1e-9 * regularizers
  loss += tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 0))

Tensor("add_13:0", shape=(), dtype=float32)


In [94]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.667179
Minibatch accuracy: 10.2%
Validation accuracy: 16.4%
Minibatch loss at step 50: 2.016844
Minibatch accuracy: 26.6%
Validation accuracy: 45.2%
Minibatch loss at step 100: 1.508590
Minibatch accuracy: 47.7%
Validation accuracy: 65.8%
Minibatch loss at step 150: 1.471087
Minibatch accuracy: 48.4%
Validation accuracy: 73.5%
Minibatch loss at step 200: 1.217636
Minibatch accuracy: 60.2%
Validation accuracy: 77.9%
Minibatch loss at step 250: 1.096534
Minibatch accuracy: 67.2%
Validation accuracy: 80.8%
Minibatch loss at step 300: 0.873980
Minibatch accuracy: 74.2%
Validation accuracy: 81.8%
Minibatch loss at step 350: 0.837062
Minibatch accuracy: 71.9%
Validation accuracy: 81.9%
Minibatch loss at step 400: 1.067672
Minibatch accuracy: 63.3%
Validation accuracy: 82.8%
Minibatch loss at step 450: 0.851678
Minibatch accuracy: 75.0%
Validation accuracy: 83.2%
Minibatch loss at step 500: 0.817031
Minibatch accuracy: 72.7%
Validation accuracy: 83.8%
M